In [9]:
# !pip install torch

In [10]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
import datetime


df = pd.read_csv('data_collection/ohlcv_data/ohlcv_auto.csv')
df.head()

,Date,Ticker,AMARAJABAT.NS,ASHOKLEY.NS,BAJAJ-AUTO.NS,BALKRISIND.NS,BHARATFORG.NS,BOSCHLTD.NS,EICHERMOT.NS,EXIDEIND.NS,HEROMOTOCO.NS,M&M.NS,MARUTI.NS,MRF.NS,TATAMOTORS.NS,TIINDIA.NS,TVSMOTOR.NS
0,1995-12-25,Adj Close,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.472771,NaN,NaN
1,1995-12-25,Close,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.819626,NaN,NaN
2,1995-12-25,High,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.819626,NaN,NaN
3,1995-12-25,Low,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.819626,NaN,NaN
4,1995-12-25,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.819626,NaN,NaN


In [11]:
df['Date'] = pd.to_datetime(df['Date'])

In [12]:
df.iloc[1937, :]

Date             1997-03-19 00:00:00
Ticker                        Volume
AMARAJABAT.NS                    NaN
ASHOKLEY.NS                      NaN
BAJAJ-AUTO.NS                    NaN
BALKRISIND.NS                    NaN
BHARATFORG.NS                 2560.0
BOSCHLTD.NS                      NaN
EICHERMOT.NS                  1000.0
EXIDEIND.NS                      NaN
HEROMOTOCO.NS                    NaN
M&M.NS                       23600.0
MARUTI.NS                        NaN
MRF.NS                           NaN
TATAMOTORS.NS              1652397.0
TIINDIA.NS                       NaN
TVSMOTOR.NS                      NaN
Name: 1937, dtype: object

In [13]:
start_mask =  df['Date'] >= datetime.datetime.fromisoformat('1996-03-19')
end_mask =  df['Date'] > datetime.datetime.fromisoformat('1997-03-19')
i_start = start_mask[start_mask].index.min()
i_end = end_mask[end_mask].index.min()
print(len(df.iloc[i_start:i_end,:]))
print(i_start, i_end)
# print(i_start, i_end, df.iloc[i_start, :], df.iloc[i_end+1, :])

1572
366 1938


In [17]:
ticker = 'ASHOKLEY.NS'
t = df[['Date', 'Ticker', ticker]].pivot_table(index='Date', columns='Ticker')
t.columns = t.columns.droplevel()
t.reset_index(inplace=True)
t.set_index('Date', inplace=True)
print(t.head(), t.index)

Ticker      Adj Close  Close   High    Low   Open    Volume
Date                                                       
2002-07-01   0.907560  5.970  6.150  5.900  6.125  711934.0
2002-07-02   0.889318  5.850  6.120  5.815  5.890  520842.0
2002-07-03   0.890838  5.860  5.925  5.820  5.850  277700.0
2002-07-04   1.454707  5.895  5.970  5.675  5.750  597586.0
2002-07-05   1.446070  5.860  6.105  5.830  5.945  425350.0 DatetimeIndex(['2002-07-01', '2002-07-02', '2002-07-03', '2002-07-04',
               '2002-07-05', '2002-07-08', '2002-07-09', '2002-07-10',
               '2002-07-11', '2002-07-12',
               ...
               '2022-03-02', '2022-03-03', '2022-03-04', '2022-03-07',
               '2022-03-08', '2022-03-09', '2022-03-10', '2022-03-11',
               '2022-03-14', '2022-03-15'],
              dtype='datetime64[ns]', name='Date', length=4899, freq=None)


In [46]:
df = pd.read_csv("data_collection/NIFTY 50.csv")
df.head()
df['Date'] = pd.to_datetime(df['Date'])
df.rename(columns={'SharesTraded' : 'Volume'}, inplace=True)
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head()

start_date = '2017-01-01'
end_date = None

if start_date is not None:
    start_mask =  df['Date'] >= datetime.datetime.fromisoformat(start_date)
    i_start = start_mask[start_mask].index.min()
else:
    i_start = 0

if end_date is not None:
    end_mask =  df['Date'] > datetime.datetime.fromisoformat(end_date)
    i_end = end_mask[end_mask].index.min()
else:
    i_end = len(df)

df.reset_index(drop=True, inplace=True)
df.set_index(['Date'], inplace=True)
df = df.iloc[i_start:i_end]
print(df.shape)
sliding_window_view(df.values, (10, 6)).squeeze().reshape((-1, 60)).shape
# print(df.shape)

(1297, 6)


(1288, 60)

In [12]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view as sliding_window_view
import pickle
import datetime


class NSEDataset(Dataset):
    def __init__(self, ohlcv_dir, target_ticker, target_ticker_file, len_window, len_corr_traceback, nP, nN, 
    keep_tickers=None, ohlcv_prefix='', ohlcv_sufix='', ohlcv_files=None, start_date=None, end_date=None,
    target_feat='c', keep_feat='ohlcva'):

        feat_name_map = {
            'o' : 'Open', 
            'h' : 'High', 
            'l' : 'Low', 
            'c' : 'Close', 
            'v' : 'Volume',
            'a' : 'Adj Close'
        }

        if ohlcv_files is not None:
            ohlcv_files = set(ohlcv_files)
        
        if keep_tickers is not None:
            keep_tickers = set(keep_tickers)
    
        temp_df = pd.read_csv(os.path.join(ohlcv_dir, target_ticker_file))
        temp_df['Date'] = pd.to_datetime(temp_df['Date'])

        if start_date is not None:
            start_mask =  temp_df['Date'] >= datetime.datetime.fromisoformat(start_date)
            i_start = start_mask[start_mask].index.min()
        else:
            i_start = 0
        
        if end_date is not None:
            end_mask =  temp_df['Date'] > datetime.datetime.fromisoformat(end_date)
            i_end = end_mask[end_mask].index.min()
        else:
            i_end = len(temp_df)

        temp_df.reset_index(drop=True, inplace=True)
        temp_df.set_index(['Date', 'Ticker'], inplace=True)
        
        self.df = temp_df.iloc[i_start:i_end]
        if ohlcv_files is not None and target_ticker_file not in ohlcv_files:
            self.df = pd.DataFrame(self.df.loc[:, target_ticker])
        
        for f in os.listdir(ohlcv_dir):
            if f.startswith(ohlcv_prefix) and f.endswith(ohlcv_sufix) and (ohlcv_files is None or f in ohlcv_files):
                if f == target_ticker_file:
                    continue
                
                temp_df = pd.read_csv(os.path.join(ohlcv_dir, f))
                temp_df.reset_index(drop=True, inplace=True)
                temp_df.set_index(['Date', 'Ticker'], inplace=True)
                self.df = self.df.merge(temp_df, how='left', left_index=True, right_index=True, suffixes=('', '_y'))
                self.df.drop(self.df.filter(regex='_y$').columns.tolist(), axis=1, inplace=True)

        if keep_tickers is not None:
            for c in self.df:
                if not (c == target_ticker or c in keep_tickers):
                    self.df.drop(c, axis=1, inplace=True)

        self.df = self.df.pivot_table(index='Date', columns='Ticker')
        self.df.columns = self.df.columns.map('_'.join)

        target_feat_name = "{}_{}".format(target_ticker, feat_name_map[target_feat])
        self.y = self.df.loc[:, target_feat_name]
        self.df.drop(target_feat_name, axis=1, inplace=True)

        drop_features = set(feat_name_map.keys()).difference({feat for feat in keep_feat})
        for feat in drop_features:
            self.df.drop(self.df.filter(regex='_{}$'.format(feat_name_map[feat])).columns.tolist(), axis=1, inplace=True)

        self.swdf = []
        for i_start in range(len_corr_traceback, len(self.df)+1):
            if i_start % 50 == 0:
                print(i_start, len(self.df))
            self.swdf.append(self.get_high_corr(self.y, self.df.iloc[i_start-len_corr_traceback:i_start, :], len_window, len_corr_traceback, nP, nN))

        self.swdf = pd.DataFrame(self.swdf)

        # Corr window + R..wip
        # self.swdf = np.squeeze(sliding_window_view(dataset.df, (len_window, len(self.df.columns))))

    def get_high_corr(self, target: pd.Series, candidates: pd.DataFrame, len_window, len_corr_traceback, nP, nN):
        # candidates.
        corr = candidates.corrwith(target)
        p_best = corr.nlargest(nP)
        n_best = corr.nsmallest(nN)
        newrow = candidates.iloc[-len_window:, candidates.columns.get_indexer(p_best.index)].melt()['value'].tolist()
        newrow.extend(candidates.iloc[-len_window:, candidates.columns.get_indexer(n_best.index)].melt()['value'].tolist())
        return newrow


def save_NSEDataset(dataset, opfile):
    PICKLE_PROTOCOL = 4  # Default since Python 3.8.
    with open(opfile, 'wb') as f:
        pickle.dump(dataset, f, PICKLE_PROTOCOL)

def load_NSEDataset(ipfile):
    PICKLE_PROTOCOL = 4
    with open(ipfile, 'rb') as f:
        dataset = pickle.load(f)
    
    return dataset 


In [13]:
dataset = NSEDataset('data_collection/ohlcv_data', 'ITC.NS', 'ohlcv_fmcg.csv', len_window=10, len_corr_traceback=20, 
nP=10, nN=10, keep_feat='o', start_date='2017-01-01')
save_NSEDataset(dataset, 'data_collection/pickled_datasets/itc_Jan17_w10_t20_p10_n10_o.pkl')

50 1286
100 1286
150 1286
200 1286
250 1286
300 1286
350 1286
400 1286
450 1286
500 1286
550 1286
600 1286
650 1286
700 1286
750 1286
800 1286
850 1286
900 1286
950 1286
1000 1286
1050 1286
1100 1286
1150 1286
1200 1286
1250 1286


In [15]:
dataset.swdf

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,249.250000,248.800003,251.600006,249.750000,254.250000,255.100006,259.600006,259.950012,263.500000,258.399994,...,825.000000,811.200012,830.000000,826.950012,805.000000,806.500000,818.799988,818.299988,825.000000,814.950012
1,1484.349976,1509.000000,1545.500000,1558.949951,1532.599976,1548.150024,1562.500000,1567.000000,1564.000000,1575.000000,...,811.200012,830.000000,826.950012,805.000000,806.500000,818.799988,818.299988,825.000000,814.950012,802.799988
2,251.600006,249.750000,254.250000,255.100006,259.600006,259.950012,263.500000,258.399994,256.200012,260.000000,...,6750.000000,6800.000000,6980.000000,7009.850098,6868.950195,6930.100098,7000.000000,7045.000000,6931.450195,6930.399902
3,249.750000,254.250000,255.100006,259.600006,259.950012,263.500000,258.399994,256.200012,260.000000,271.000000,...,6800.000000,6980.000000,7009.850098,6868.950195,6930.100098,7000.000000,7045.000000,6931.450195,6930.399902,7028.000000
4,254.250000,255.100006,259.600006,259.950012,263.500000,258.399994,256.200012,260.000000,271.000000,274.000000,...,6980.000000,7009.850098,6868.950195,6930.100098,7000.000000,7045.000000,6931.450195,6930.399902,7028.000000,6981.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,216.800003,210.000000,211.500000,213.199997,214.399994,216.050003,217.699997,222.000000,223.899994,227.699997,...,925.000000,885.000000,887.000000,930.000000,945.000000,983.000000,980.900024,911.500000,912.849976,916.799988
1263,210.000000,211.500000,213.199997,214.399994,216.050003,217.699997,222.000000,223.899994,227.699997,231.000000,...,875.000000,897.900024,918.299988,880.000000,873.950012,850.000000,845.000000,850.000000,859.700012,892.000000
1264,211.500000,213.199997,214.399994,216.050003,217.699997,222.000000,223.899994,227.699997,231.000000,231.899994,...,690.000000,699.500000,711.950012,712.150024,688.400024,659.000000,659.000000,685.000000,705.549988,724.299988
1265,213.199997,214.399994,216.050003,217.699997,222.000000,223.899994,227.699997,231.000000,231.899994,236.250000,...,918.299988,880.000000,873.950012,850.000000,845.000000,850.000000,859.700012,892.000000,919.000000,911.299988


In [43]:
len(dataset.swdf)

6580